In [1]:
import os
import random
import time
from typing import List, Tuple
from urllib.request import urlopen

import numpy as np
import scipy
import pandas as pd
import sklearn
import requests
import matplotlib.pyplot as plt

In [2]:
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor

In [3]:
!python --version

Python 3.8.5


In [12]:
import os
import random
import time
import json
from typing import Tuple
from urllib.request import urlopen

# ゲームサーバのアドレス
GAME_SERVER = os.getenv('GAME_SERVER', 'https://contest.2021-spring.gbc.tenka1.klab.jp')
# あなたのトークン
TOKEN = "b16c91e37d4c75777beba2eb755056fd"
# GameAPIを呼ぶ際のインターバル
GAME_INFO_SLEEP_TIME = 5000


def call_api(x: str) -> bytes:
    with urlopen(f'{GAME_SERVER}{x}') as res:
        return res.read()


def call_move(index: int, x: int, y: int) -> dict:
    res_str = call_api(f'/api/move/{TOKEN}/{index}-{x}-{y}')
    res_json = json.loads(res_str)
    return res_json


def call_move_next(index: int, x: int, y: int) -> dict:
    res_str = call_api(f'/api/move_next/{TOKEN}/{index}-{x}-{y}')
    res_json = json.loads(res_str)
    return res_json


def call_game() -> dict:
    res_str = call_api(f'/api/game/{TOKEN}')
    res_json = json.loads(res_str)
    return res_json


def call_master_data() -> dict:
    res_str = call_api(f'/api/master_data')
    res_json = json.loads(res_str)
    return res_json


class Bot:
    def __init__(self):
        self.move_histories = [["H", "O", "G", "E"] for _ in range(5)]

        self.master_data = call_master_data()
        self.game_info = call_game()
        self.start_game_time_ms = self.game_info['now']
        print("Start:", self.start_game_time_ms)
        self.start_time_ms = time.perf_counter_ns() // 1000000
        self.next_call_game_info_time_ms = self.get_now_game_time_ms() + GAME_INFO_SLEEP_TIME
        self.agent_move_finish_ms = [0] * self.master_data['num_agent']
        self.agent_move_point_queue = [[] for _ in range(self.master_data['num_agent'])]
        self.agent_last_point = [[] for _ in range(self.master_data['num_agent'])]
        for i in range(self.master_data['num_agent']):
            agent_move = self.game_info['agent'][i]['move']
            self.agent_last_point[i] = [agent_move[-1]['x'], agent_move[-1]['y']]
            self.set_move_point(i)

    # タスクを取得
    def choice_task(self, index: int) -> str:
        tasks = self.game_info['task'][42:]
        best_efficiency = 0
        best_task = ["Z"]
        for task1 in tasks:
            list_task1 = list(task1["s"])
            point = (1.0 / (task1["total"] + 1.0)) ** 1.0
            tim = 0
            if self.move_histories[index][-4:] == list_task1[:4]:
                list_task1 = list_task1[4:]
            elif self.move_histories[index][-3:] == list_task1[:3]:
                list_task1 = list_task1[3:]
            elif self.move_histories[index][-2:] == list_task1[:2]:
                list_task1 = list_task1[2:]
            elif self.move_histories[index][-1:] == list_task1[:1]:
                list_task1 = list_task1[1:]
            else:
                pass
            c = self.move_histories[index][-1]
            x, y = self.get_checkpoint(c)
            for next_c in list_task1:
                next_x, next_y = self.get_checkpoint(next_c)
                t = max(1.0, 0.1 * ((x-next_x)**2 + (y-next_y)**2)**0.5)
                tim += t

            efficiency = point / tim
            if best_efficiency < efficiency:
                best_efficiency = efficiency
                best_task = list_task1

        if random.random() < 0.05:
            best_task = list(random.choice(tasks)["s"])
        return "".join(best_task)

    def get_now_game_time_ms(self) -> int:
        now_ms = time.perf_counter_ns() // 1000000
        return self.start_game_time_ms + (now_ms - self.start_time_ms)

    def get_checkpoint(self, name: str) -> Tuple[int, int]:
        index = ord(name) - ord('A')
        checkpoint = self.master_data['checkpoints'][index]
        return checkpoint['x'], checkpoint['y']

    # 移動予定を設定
    def set_move_point(self, index: int):
        next_task = self.choice_task(index)
        print("Agent#{} next task:{}".format(index, next_task))
        for i in range(len(next_task)):
            before_x = self.agent_last_point[index][0]
            before_y = self.agent_last_point[index][1]
            move_x, move_y = self.get_checkpoint(next_task[i])

            # 移動先が同じ場所の場合判定が入らないため別の箇所に移動してからにする
            if move_x == before_x and move_y == before_y:
                tmp_x = (before_x-1) if before_x > 10 else (before_x+1)
                tmp_y = before_y
                self.agent_move_point_queue[index].append([tmp_x, tmp_y])

            self.agent_move_point_queue[index].append([move_x, move_y])
            self.move_histories[index].append(next_task[i])
            self.move_histories[index] = self.move_histories[index][1:]
            self.agent_last_point[index] = [move_x, move_y]

    def move_next(self, index: int) -> dict:
        move_next_point = self.agent_move_point_queue[index].pop(0)
        move_next_res = call_move_next(index+1, move_next_point[0], move_next_point[1])
        print("Agent#{} move_next to ({}, {})".format(index+1, move_next_point[0], move_next_point[1]))

        if move_next_res['status'] != 'ok':
            print(move_next_res)
            exit(1)

        assert len(move_next_res['move']) > 1

        self.agent_move_finish_ms[index] = move_next_res['move'][1]['t'] + 100

        # タスクを全てやりきったら次のタスクを取得
        if not self.agent_move_point_queue[index]:
            self.set_move_point(index)

        return move_next_res

    # game_infoの状態でのスコアを取得
    def get_now_score(self) -> float:
        score = 0.0
        tasks = self.game_info['task']
        for i in range(len(tasks)):
            if tasks[i]['total'] == 0:
                continue
            score += tasks[i]['weight'] * tasks[i]['count'] / tasks[i]['total']

        return score

    def solve(self):
        while True:
            now_game_time_ms = self.get_now_game_time_ms()

            # エージェントを移動させる
            for i in range(self.master_data['num_agent']):
                if self.agent_move_finish_ms[i] < now_game_time_ms:
                    move_next_res = self.move_next(i)
                    # 次の移動予定がない場合もう一度実行する
                    if len(move_next_res['move']) == 2:
                        self.move_next(i)

            # ゲーム情報更新
            if self.next_call_game_info_time_ms < now_game_time_ms:
                print('Update GameInfo')
                self.game_info = call_game()
                self.next_call_game_info_time_ms = self.get_now_game_time_ms() + GAME_INFO_SLEEP_TIME
                print("Score: {}".format(self.get_now_score()))

            time.sleep(0.5)


In [13]:
bot = Bot()
bot.solve()

Start: 8775203
Agent#0 next task:MXCOAEZIMD
Agent#1 next task:MXCOAEZIMD
Agent#2 next task:MXCOAEZIMD
Agent#3 next task:MXCOAEZIMD
Agent#4 next task:MXCOAEZIMD
Agent#1 move_next to (3, 0)
Agent#2 move_next to (4, 0)
Agent#3 move_next to (3, 0)
Agent#4 move_next to (3, 0)
Agent#5 move_next to (3, 0)
Agent#2 move_next to (3, 0)
Agent#2 move_next to (21, 0)
Agent#4 move_next to (21, 0)
Agent#3 move_next to (21, 0)
Agent#1 move_next to (21, 0)
Agent#5 move_next to (21, 0)
Agent#2 move_next to (17, 4)
Agent#2 move_next to (12, 15)
Agent#1 move_next to (17, 4)
Agent#3 move_next to (17, 4)
Agent#2 move_next to (12, 7)
Agent#4 move_next to (17, 4)
Agent#5 move_next to (17, 4)
Agent#1 move_next to (12, 15)
Agent#2 move_next to (25, 9)
Agent#3 move_next to (12, 15)
Update GameInfo
Score: 8270.95106131327
Agent#1 move_next to (12, 7)
Agent#2 move_next to (10, 20)
Agent#3 move_next to (12, 7)
Agent#4 move_next to (12, 15)
Agent#5 move_next to (12, 15)
Agent#1 move_next to (25, 9)
Agent#3 move_next

Agent#1 move_next to (12, 7)
Agent#4 move_next to (10, 20)
Agent#5 move_next to (10, 20)
Agent#3 move_next to (3, 0)
Agent#1 move_next to (17, 4)
Agent#2 move_next to (10, 20)
Agent#3 move_next to (8, 16)
Update GameInfo
Score: 8303.367972417545
Agent#1 move_next to (3, 20)
Agent#3 move_next to (5, 24)
Agent#5 move_next to (15, 19)
Agent#1 move_next to (0, 28)
Agent#0 next task:ZTHZUBSUDO
Agent#4 move_next to (15, 19)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (5, 24)
Agent#5 move_next to (5, 24)
Agent#1 move_next to (10, 20)
Agent#3 move_next to (12, 7)
Agent#4 move_next to (0, 28)
Agent#5 move_next to (0, 28)
Agent#3 move_next to (17, 4)
Agent#1 move_next to (30, 27)
Agent#2 move_next to (5, 24)
Agent#2 move_next to (0, 28)
Agent#4 move_next to (15, 19)
Agent#5 move_next to (15, 19)
Agent#4 move_next to (3, 20)
Agent#5 move_next to (3, 20)
Update GameInfo
Score: 8295.014034727668
Agent#1 move_next to (1, 10)
Agent#3 move_next to (3, 20)
Agent#2 move_next to (15, 19)
Agent#2 m

Agent#4 move_next to (30, 27)
Agent#2 move_next to (10, 20)
Agent#3 move_next to (10, 20)
Agent#5 move_next to (30, 27)
Agent#1 move_next to (15, 19)
Agent#1 move_next to (3, 20)
Agent#3 move_next to (30, 27)
Agent#2 move_next to (30, 27)
Agent#4 move_next to (1, 10)
Agent#5 move_next to (1, 10)
Agent#3 move_next to (1, 10)
Agent#2 move_next to (1, 10)
Agent#1 move_next to (12, 15)
Agent#0 next task:MXCOAEZIMD
Agent#4 move_next to (10, 20)
Agent#5 move_next to (10, 20)
Update GameInfo
Score: 8313.049222510714
Agent#1 move_next to (3, 0)
Agent#3 move_next to (10, 20)
Agent#2 move_next to (10, 20)
Agent#1 move_next to (21, 0)
Agent#1 move_next to (17, 4)
Agent#4 move_next to (15, 19)
Agent#5 move_next to (15, 19)
Agent#3 move_next to (15, 19)
Agent#4 move_next to (5, 24)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (0, 28)
Agent#5 move_next to (5, 24)
Agent#5 move_next to (0, 28)
Agent#1 move_next to (12, 15)
Agent#1 move_next to (12, 7)
Agent#3 move_next to (5, 24)
Agent#3 move_ne

Agent#5 move_next to (21, 23)
Agent#2 move_next to (9, 29)
Update GameInfo
Score: 8312.989497879704
Agent#3 move_next to (29, 30)
Agent#2 next task:BEZJLJ
Agent#4 move_next to (0, 28)
Agent#3 next task:CBUOSPS
Agent#1 move_next to (12, 15)
Agent#2 move_next to (29, 30)
Agent#1 next task:BEZJLJ
Agent#5 move_next to (0, 28)
Agent#4 next task:CBUOSPS
Agent#1 move_next to (0, 28)
Agent#1 move_next to (21, 23)
Agent#3 move_next to (5, 24)
Agent#4 move_next to (17, 4)
Agent#2 move_next to (5, 24)
Agent#5 move_next to (17, 4)
Agent#3 move_next to (25, 9)
Agent#1 move_next to (0, 28)
Agent#0 next task:CBUOSPS
Agent#4 move_next to (5, 24)
Agent#2 move_next to (25, 9)
Agent#5 move_next to (5, 24)
Update GameInfo
Score: 8319.824481733407
Agent#1 move_next to (17, 4)
Agent#3 move_next to (10, 20)
Agent#2 move_next to (10, 20)
Agent#4 move_next to (15, 19)
Agent#5 move_next to (15, 19)
Agent#1 move_next to (5, 24)
Agent#3 move_next to (29, 30)
Agent#2 move_next to (29, 30)
Agent#4 move_next to (12,

Agent#2 move_next to (15, 11)
Agent#4 move_next to (29, 13)
Agent#4 move_next to (15, 11)
Agent#5 move_next to (29, 13)
Update GameInfo
Score: 9069.48869237724
Agent#1 move_next to (5, 24)
Agent#1 move_next to (7, 5)
Agent#5 move_next to (15, 11)
Agent#3 move_next to (14, 11)
Agent#2 move_next to (14, 11)
Agent#4 move_next to (14, 11)
Agent#1 move_next to (30, 27)
Agent#0 next task:MXCOAEZIMD
Agent#3 move_next to (15, 11)
Agent#3 move_next to (7, 5)
Agent#5 move_next to (14, 11)
Agent#2 move_next to (15, 11)
Agent#2 move_next to (7, 5)
Agent#4 move_next to (15, 11)
Agent#4 move_next to (7, 5)
Agent#3 move_next to (5, 24)
Agent#3 move_next to (7, 5)
Agent#5 move_next to (15, 11)
Agent#5 move_next to (7, 5)
Agent#1 move_next to (3, 0)
Agent#2 move_next to (5, 24)
Agent#2 move_next to (7, 5)
Agent#4 move_next to (5, 24)
Agent#4 move_next to (7, 5)
Agent#5 move_next to (5, 24)
Agent#5 move_next to (7, 5)
Update GameInfo
Score: 8993.513605906473
Agent#3 move_next to (30, 27)
Agent#2 next ta

...

Agent#3 move_next to (9, 29)
Agent#2 move_next to (24, 17)
Agent#1 next task:JUJTLQI
Agent#3 move_next to (15, 11)
Agent#4 move_next to (15, 19)
Agent#1 move_next to (30, 27)
Agent#4 move_next to (29, 30)
Agent#5 move_next to (9, 29)
Agent#5 move_next to (15, 11)
Update GameInfo
Score: 9258.50520250034
Agent#1 move_next to (9, 29)
Agent#2 move_next to (29, 30)
Agent#3 move_next to (24, 17)
Agent#2 next task:JUJTLQI
Agent#1 move_next to (15, 11)
Agent#4 move_next to (30, 27)
Agent#2 move_next to (15, 19)
Agent#5 move_next to (24, 17)
Agent#4 next task:JUJTLQI
Agent#3 move_next to (29, 30)
Agent#2 move_next to (29, 30)
Agent#1 move_next to (24, 17)
Agent#0 next task:JUJTLQI
Agent#3 move_next to (15, 19)
Agent#4 move_next to (9, 29)
Agent#4 move_next to (15, 11)
Agent#5 move_next to (29, 30)
Agent#2 move_next to (30, 27)
Agent#3 move_next to (29, 30)
Agent#1 move_next to (29, 30)
Agent#5 move_next to (15, 19)
Agent#4 move_next to (24, 17)
Agent#3 next task:JUJTLQI
Update GameInfo
Score: 9

Agent#1 move_next to (24, 17)
Agent#0 next task:JUJTLQI
Agent#4 move_next to (9, 29)
Agent#4 move_next to (15, 11)
Agent#5 move_next to (29, 30)
Agent#2 move_next to (30, 27)
Agent#3 move_next to (29, 30)
Agent#5 move_next to (15, 19)
Agent#1 move_next to (29, 30)
Agent#3 move_next to (15, 19)
Agent#4 move_next to (24, 17)
Agent#3 next task:JUJTLQI
Agent#1 move_next to (15, 19)
Agent#5 move_next to (29, 30)
Agent#2 move_next to (9, 29)
Agent#2 move_next to (15, 11)
Agent#3 move_next to (29, 30)
Update GameInfo
Score: 9250.505447469117
Agent#1 move_next to (29, 30)
Agent#4 move_next to (29, 30)
Agent#5 move_next to (30, 27)
Agent#3 move_next to (30, 27)
Agent#2 move_next to (24, 17)
Agent#1 next task:JUJTLQI
Agent#4 move_next to (15, 19)
Agent#1 move_next to (30, 27)
Agent#5 move_next to (9, 29)
Agent#5 move_next to (15, 11)
Agent#3 move_next to (9, 29)
Agent#4 move_next to (29, 30)
Agent#3 move_next to (15, 11)
Agent#1 move_next to (9, 29)
Agent#1 move_next to (15, 11)
Agent#2 move_nex

Agent#2 move_next to (24, 17)
Agent#4 move_next to (29, 30)
Agent#3 move_next to (15, 19)
Agent#1 move_next to (15, 11)
Agent#1 move_next to (7, 5)
Agent#2 move_next to (3, 0)
Agent#4 move_next to (30, 27)
Agent#5 move_next to (24, 17)
Agent#4 next task:JUJTLQI
Agent#1 move_next to (5, 24)
Agent#1 move_next to (7, 5)
Agent#3 move_next to (12, 15)
Agent#2 move_next to (3, 20)
Agent#1 next task:UOSPSAF
Update GameInfo
Score: 9244.533074329936
Agent#4 move_next to (9, 29)
Agent#4 move_next to (15, 11)
Agent#5 move_next to (29, 30)
Agent#3 move_next to (0, 28)
Agent#5 move_next to (15, 19)
Agent#1 move_next to (30, 27)
Agent#0 next task:ZTINMQS
Agent#3 move_next to (21, 23)
Agent#2 move_next to (15, 19)
Agent#4 move_next to (24, 17)
Agent#3 next task:JUJTLQI
Agent#5 move_next to (29, 30)
Agent#1 move_next to (10, 20)
Agent#3 move_next to (0, 28)
Agent#2 move_next to (12, 15)
Agent#5 move_next to (30, 27)
Agent#4 move_next to (29, 30)
Agent#2 move_next to (0, 28)
Agent#3 move_next to (12, 7

Agent#4 move_next to (28, 18)
Agent#1 move_next to (29, 13)
Agent#2 move_next to (29, 30)
Agent#5 move_next to (15, 19)
Agent#3 move_next to (15, 19)
Update GameInfo
Score: 9230.332382715613
Agent#2 move_next to (9, 29)
Agent#5 move_next to (29, 30)
Agent#1 move_next to (8, 16)
Agent#1 move_next to (21, 0)
Agent#4 move_next to (29, 13)
Agent#3 move_next to (29, 30)
Agent#5 move_next to (30, 27)
Agent#2 move_next to (29, 30)
Agent#1 next task:UJTLQI
Agent#3 move_next to (30, 27)
Agent#1 move_next to (30, 6)
Agent#0 next task:ZKDGNRXK
Agent#4 move_next to (8, 16)
Agent#4 move_next to (21, 0)
Agent#2 move_next to (15, 19)
Agent#5 move_next to (9, 29)
Agent#5 move_next to (15, 11)
Agent#3 move_next to (9, 29)
Agent#3 move_next to (15, 11)
Update GameInfo
Score: 9222.679868421186
Agent#1 move_next to (10, 20)
Agent#4 move_next to (30, 6)
Agent#3 next task:ZKDGNRXK
Agent#1 move_next to (30, 6)
Agent#2 move_next to (29, 30)
Agent#5 move_next to (24, 17)
Agent#4 next task:WGXZGV
Agent#3 move_n

KeyboardInterrupt: 

In [6]:
random.random()

0.9575105958911774

In [14]:
def call_game() -> dict:
    res_str = call_api(f'/api/game/{TOKEN}')
    res_json = json.loads(res_str)
    return res_json

In [16]:
g = call_game()
g

{'status': 'ok',
 'now': 10598173,
 'agent': [{'move': [{'x': 15, 'y': 24, 't': 10597918},
    {'x': 0, 'y': 28, 't': 10599471}],
   'history': 'LYCVSRLYCV',
   'history_times': [10582152,
    10583902,
    10585875,
    10587885,
    10589438,
    10590881,
    10592185,
    10593935,
    10595908,
    10597918]},
  {'move': [{'x': 9, 'y': 29, 't': 10596952},
    {'x': 0, 'y': 14, 't': 10598702},
    {'x': 17, 'y': 4, 't': 10600675}],
   'history': 'OVIAJOVSRL',
   'history_times': [10582921,
    10583870,
    10585011,
    10586574,
    10589435,
    10591703,
    10592652,
    10594205,
    10595648,
    10596952]},
  {'move': [{'x': 24, 'y': 17, 't': 10597305},
    {'x': 12, 'y': 7, 't': 10598868},
    {'x': 29, 'y': 30, 't': 10601729}],
   'history': 'LYCVIAJOVI',
   'history_times': [10581649,
    10583399,
    10585372,
    10587382,
    10588523,
    10590086,
    10592947,
    10595215,
    10596164,
    10597305]},
  {'move': [{'x': 12, 'y': 7, 't': 10598068},
    {'x': 29, '

In [21]:
for task in g["task"][42:]:
    s = task["s"]
    total = task["total"]
    print(f"{len(s):2d} {1 / total / len(s):.8f}")

 6 0.00003737
 7 0.00004580
 6 0.00005482
10 0.00003902
10 0.00004257
 8 0.00004659
 7 0.00003315
 6 0.00003813
 9 0.00004559
10 0.00004771
10 0.00005247
 5 0.00003709
 9 0.00004257
10 0.00004766
 8 0.00006717
 5 0.00005111
10 0.00004243
 8 0.00004775
 7 0.00002409
 7 0.00004579
10 0.00003512
 8 0.00006805
 6 0.00006004
 6 0.00005051
10 0.00005277
10 0.00005565
 8 0.00005687
 9 0.00005048
 9 0.00005482
 6 0.00006125
 7 0.00002629
 8 0.00005268
 6 0.00004971
 6 0.00005120
10 0.00004973
 5 0.00005905
 5 0.00004134
 9 0.00005888
 7 0.00005305
 7 0.00006812
 7 0.00005471
 5 0.00004151
 9 0.00005885
 6 0.00006513
 8 0.00005487
 8 0.00003798
 6 0.00007044
 6 0.00007332
10 0.00005491
 7 0.00003690
 7 0.00005077
10 0.00005025
 5 0.00005070
 6 0.00004759
 9 0.00006346
 5 0.00002754
 7 0.00005609
 7 0.00005507
 7 0.00005561
 7 0.00005889
 7 0.00006529
 8 0.00006320
 7 0.00006066
 9 0.00006239
 6 0.00006792
 7 0.00004589
10 0.00011682
 8 0.00010647
 6 0.00010314
 9 0.00010307
 6 0.00007959
 5 0.0